In [89]:
import pandas as pd

## US Languages Dataset

In [90]:
filename = "2009-2013-acs-lang-tables-nation (1).csv"

# skip the first lines (the header)
us_lang_data = pd.read_csv(filename, skiprows = 4)

# cut off the last lines that contains the footers
us_lang_data = us_lang_data[:372]

# rename columns and drop unwanted
us_lang_data = us_lang_data.rename(columns={
    "Unnamed: 0": "languages",
    "Number of speakers1": "num_speakers",
    'Speak English less than "Very Well"1' : "low_eng_prof"
    })

us_lang_data = us_lang_data.drop(columns=['Margin of Error2', 'Margin of Error2.1'])

# retrieve total number of speakers for percentage calculations
row = us_lang_data[us_lang_data['languages'] == ".Population 5 years and over"]
cnt = row["num_speakers"].item()
pop_cnt = int(cnt.replace(",", ""))

# create new variables or modify
def become_numeric(lst, is_perc):
  acc = []
  for val in lst:
    # val is small number
    if val.isnumeric():
      val = int(val)
      if is_perc:
        perc = val / pop_cnt
        acc.append(perc)
      else:
        acc.append(val)
    # val is a number with commas
    elif val.find(",") != -1:
      val = val.replace(",", "")
      val = int(val)
      if is_perc:
        perc = val / pop_cnt
        acc.append(perc)
      else:
        acc.append(val)
    # val is something like (D)
    elif val.find("(") != -1:
      acc.append(-1)
    else:
      acc.append("ISSUE")
  return acc

# percentage of the population that speaks that language
us_lang_data["speaker_perc"] = become_numeric(us_lang_data["num_speakers"], True)
us_lang_data["speaker_perc"].astype(float)

# percentage of the language population that speaks English less than very well
us_lang_data["low_prof_perc"] = become_numeric(us_lang_data["low_eng_prof"], True)
us_lang_data["low_prof_perc"].astype(float)

# number of speakers
us_lang_data["num_speakers"] = become_numeric(us_lang_data["num_speakers"], False)
us_lang_data["num_speakers"].astype(int)

# low english proficiency (population that speaks English less than very well)
us_lang_data["low_eng_prof"] = become_numeric(us_lang_data["low_eng_prof"], False)
us_lang_data["low_eng_prof"].astype(int)

us_lang_data

,languages,num_speakers,low_eng_prof,speaker_perc,low_prof_perc
0,.Population 5 years and over,291484482,25148900,1.000000e+00,8.627869e-02
1,Speak only English at home,231122908,-1,7.929167e-01,-1.000000e+00
2,Speak a language other than English at home,60361574,25148900,2.070833e-01,8.627869e-02
3,SPANISH AND SPANISH CREOLE,37458624,16344473,1.285098e-01,5.607322e-02
4,..Spanish,37458470,16344440,1.285093e-01,5.607311e-02
...,...,...,...,...,...
367,..Quechua,1300,1000,4.459929e-06,3.430714e-06
368,..Arawakian,3150,925,1.080675e-05,3.173411e-06
369,..Chibchan,1095,865,3.756632e-06,2.967568e-06
370,..Tupi-Guarani,245,155,8.405250e-07,5.317607e-07


In [91]:
# creating dictionaries that track the index
overall_langs = us_lang_data["languages"].to_list()[:3]

all_lang = us_lang_data["languages"].to_list()[3:]
major_groups = {}
subgroups = {}
langs = {}
for i in range(len(all_lang)):
  lang = all_lang[i]
  if lang.isupper():
    major_groups[lang] = i + 3
  # beginning is a . (no period follows up) and includes either "incl." or "languages"
  elif (lang[0] == "." and lang[1] != ".") and (lang.find("incl.") != -1 or lang.find("languages") != -1):
    subgroups[lang] = i + 3
  else:
    langs[lang] = i + 3


In [92]:
print(overall_langs)
print(major_groups)
print(subgroups)
print(langs)

['.Population 5 years and over', 'Speak only English at home', 'Speak a language other than English at home']
{'SPANISH AND SPANISH CREOLE': 3, 'OTHER INDO-EUROPEAN LANGUAGES': 7, 'ASIAN AND PACIFIC ISLAND LANGUAGES': 89, 'ALL OTHER LANGUAGES': 175}
{'.French (incl. Patois, Cajun)': 8, '.Portuguese (incl. Portuguese Creole)': 14, '.German (incl. Luxembourgian)': 17, '.Other West Germanic languages': 21, '.Scandinavian languages': 26, '.Serbo-Croatian languages': 35, '.Other Slavic languages': 39, '.Other Indic languages': 53, '.Other Indo-European languages': 68, '.Chinese (incl. Cantonese, Mandarin, other Chinese languages4)': 90, '.Other Asian languages': 106, '.Other Pacific Island languages': 132, '.Other Native North American languages': 177, '.African languages': 331, '.Other and unspecified languages': 352}
{'..Spanish': 4, '..Ladino': 5, '..Pachuco': 6, '..French': 9, '..Patois': 10, '..Cajun': 11, '.French Creole': 12, '.Italian': 13, '..Portuguese': 15, '..Papia Mentae': 16, 

In [93]:
group_labels = []
subgroup_labels = []
group_tracker = None
subgroup_tracker = None
for i in range(len(us_lang_data["languages"])):
  lang = us_lang_data["languages"][i]
  if lang in overall_langs:
    group_labels.append("header")
    subgroup_labels.append("header")
  elif lang in major_groups:
    group_labels.append("group_header")
    subgroup_labels.append("group_header")
    group_tracker = lang
    subgroup_tracker = None
  elif lang in subgroups:
    group_labels.append(group_tracker)
    subgroup_labels.append("subgroup_header")
    subgroup_tracker = lang
  elif lang in langs and lang[1] == ".":
    group_labels.append(group_tracker)
    subgroup_labels.append(subgroup_tracker)
  # not in a sub group
  elif lang in langs and lang[1] != ".":
    group_labels.append(group_tracker)
    subgroup_tracker = None
    subgroup_labels.append(subgroup_tracker)

In [94]:
us_lang_data["group_labels"] = group_labels
us_lang_data["subgroup_labels"] = subgroup_labels

In [95]:
us_lang_data

,languages,num_speakers,low_eng_prof,speaker_perc,low_prof_perc,group_labels,subgroup_labels
0,.Population 5 years and over,291484482,25148900,1.000000e+00,8.627869e-02,header,header
1,Speak only English at home,231122908,-1,7.929167e-01,-1.000000e+00,header,header
2,Speak a language other than English at home,60361574,25148900,2.070833e-01,8.627869e-02,header,header
3,SPANISH AND SPANISH CREOLE,37458624,16344473,1.285098e-01,5.607322e-02,group_header,group_header
4,..Spanish,37458470,16344440,1.285093e-01,5.607311e-02,SPANISH AND SPANISH CREOLE,None
...,...,...,...,...,...,...,...
367,..Quechua,1300,1000,4.459929e-06,3.430714e-06,ALL OTHER LANGUAGES,.Other and unspecified languages
368,..Arawakian,3150,925,1.080675e-05,3.173411e-06,ALL OTHER LANGUAGES,.Other and unspecified languages
369,..Chibchan,1095,865,3.756632e-06,2.967568e-06,ALL OTHER LANGUAGES,.Other and unspecified languages
370,..Tupi-Guarani,245,155,8.405250e-07,5.317607e-07,ALL OTHER LANGUAGES,.Other and unspecified languages


In [62]:
us_lang_data.to_csv('us_lang_data.csv')

## State Languages Datasets

In [99]:
all_locations = {
    'alabama' : "AL",
    'alaska' : "AK",
    'arizona' : "AZ",
    'arkansas' : "AR",
    'california' : "CA",
    'colorado' : "CO",
    'connecticut' : "CT",
    'delaware' : "DE",
    'district-columbia' : "DC",
    'florida': "FL",
    'georgia': "GA",
    'hawaii': "HI",
    'idaho' : "ID",
    'illinois' : "IL",
    'indiana' : "IN",
    'iowa' : "IA",
    'kansas' : "KS",
    'kentucky' : "KY",
    'louisiana' : "LA",
    'maine' : "ME",
    'maryland' : "MD",
    'massachusetts' : "MA",
    'michigan' : "MI",
    'minnesota' : "MN",
    'mississippi' : "MS",
    'missouri' : "MO",
    'montana' : "MT",
    'nebraska' : "NE",
    'nevada' : "NV",
    'new-hampshire' : "NH",
    'new-jersey' : "NJ",
    'new-mexico' : "NM",
    'new-york' : "NY",
    'north-carolina' : "NC",
    'north-dakota' : "ND",
    'ohio' : "OH",
    'oklahoma' : "OK",
    'oregon' : "OR",
    'pennsylvania' : "PA",
    'puerto-rico' : "PR",
    'rhode-island' : "RI",
    'south-carolina' : "SC",
    'south-dakota' : "SD",
    'tennessee' : "TN",
    'texas' : "TX",
    'utah' : "UT",
    'vermont' : "VT",
    'virginia' : "VA",
    'washington' : "WA",
    'west-virginia' : "WV",
    'wisconsin' : "WI",
    'wyoming' : "WY"
}

dfs_to_concat = []

for state in all_locations.keys():
  csvname = "2009-2013-acs-lang-tables-" + state + ".csv"
  st_lang_data = pd.read_csv(csvname, skiprows = 4)
  footer_size = 12
  length = len(st_lang_data)
  st_lang_data = st_lang_data[:length - footer_size]

  # rename columns and drop unwanted
  st_lang_data = st_lang_data.rename(columns={
      "Unnamed: 0": "languages",
      "Number of speakers1": "num_speakers",
      'Speak English less than "Very Well"1' : "low_eng_prof"
      })

  st_lang_data = st_lang_data.drop(columns=['Margin of Error2', 'Margin of Error2.1'])

  # retrieve total number of speakers for percentage calculations
  row = st_lang_data[st_lang_data['languages'] == ".Population 5 years and over"]
  cnt = row["num_speakers"].item()
  pop_cnt = int(cnt.replace(",", ""))

  # percentage of the population that speaks that language
  st_lang_data["speaker_perc"] = become_numeric(st_lang_data["num_speakers"], True)
  st_lang_data["speaker_perc"].astype(float)

  # percentage of the language population that speaks English less than very well
  st_lang_data["low_prof_perc"] = become_numeric(st_lang_data["low_eng_prof"], True)
  st_lang_data["low_prof_perc"].astype(float)

  # number of speakers
  st_lang_data["num_speakers"] = become_numeric(st_lang_data["num_speakers"], False)
  st_lang_data["num_speakers"].astype(int)

  # low english proficiency (population that speaks English less than very well)
  st_lang_data["low_eng_prof"] = become_numeric(st_lang_data["low_eng_prof"], False)
  st_lang_data["low_eng_prof"].astype(int)

  group_labels = []
  subgroup_labels = []
  group_tracker = None
  subgroup_tracker = None
  for i in range(len(st_lang_data["languages"])):
    lang = st_lang_data["languages"][i]
    if lang in overall_langs:
      group_labels.append("header")
      subgroup_labels.append("header")
    elif lang in major_groups:
      group_labels.append("group_header")
      subgroup_labels.append("group_header")
      group_tracker = lang
      subgroup_tracker = None
    elif lang in subgroups:
      group_labels.append(group_tracker)
      subgroup_labels.append("subgroup_header")
      subgroup_tracker = lang
    elif lang in langs and lang[1] == ".":
      group_labels.append(group_tracker)
      subgroup_labels.append(subgroup_tracker)
    # not in a sub group
    elif lang in langs and lang[1] != ".":
      group_labels.append(group_tracker)
      subgroup_tracker = None
      subgroup_labels.append(subgroup_tracker)

  st_lang_data["group_labels"] = group_labels
  st_lang_data["subgroup_labels"] = subgroup_labels
  st_lang_data["state"] = [state] * len(group_labels)
  st_lang_data["abbr"] = [all_locations[state]] * len(group_labels)

  dfs_to_concat.append(st_lang_data)

In [101]:
states_lang_data = pd.concat(dfs_to_concat)

In [102]:
states_lang_data

,languages,num_speakers,low_eng_prof,speaker_perc,low_prof_perc,group_labels,subgroup_labels,state,abbr
0,.Population 5 years and over,4497352,107436,1.000000,0.023889,header,header,alabama,AL
1,Speak only English at home,4261611,-1,0.947582,-1.000000,header,header,alabama,AL
2,Speak a language other than English at home,235741,107436,0.052418,0.023889,header,header,alabama,AL
3,SPANISH AND SPANISH CREOLE,151384,78970,0.033661,0.017559,group_header,group_header,alabama,AL
4,..Spanish,151385,78970,0.033661,0.017559,SPANISH AND SPANISH CREOLE,None,alabama,AL
...,...,...,...,...,...,...,...,...,...
118,..Bantu,75,-1,0.000141,-1.000000,ALL OTHER LANGUAGES,.African languages,wyoming,WY
119,"..Kru, Ibo, Yoruba",115,50,0.000217,0.000094,ALL OTHER LANGUAGES,.African languages,wyoming,WY
120,.Other and unspecified languages,80,4,0.000151,0.000008,ALL OTHER LANGUAGES,subgroup_header,wyoming,WY
121,..Finnish,20,-1,0.000038,-1.000000,ALL OTHER LANGUAGES,.Other and unspecified languages,wyoming,WY


In [103]:
states_lang_data.to_csv('states_lang_data.csv')